In [1]:
import numpy as np
import pandas as pd
import os
from config import feature_names, alpha_names, beta_names, mp, pattern, basic_names

In [15]:
file_path = "/home/tupm/datasets/thalas/2704/2. 98 sample/15000346.xlsx"
df = pd.read_excel(file_path)
if 'Unnamed: 0' == df.columns[0] and 'x' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)
    df.columns.values[0] = 'Mã XN'

In [16]:
if 'Tên xét nghiệm' in df.columns:
    df.drop(columns=['Tên xét nghiệm'], inplace=True)
df = df.set_index('Mã XN').transpose()
obj = pattern.copy()
df.columns.values

array(['1001URE', '1002GLU', '1003CRE', '1005BILT', '1006BILD',
       '1007BILC', '1012FE', '1021GOT', '1022GPT', '1027LDH', '1075SAT',
       '1092NA', '1093K', '1094CL', '220TBMNV', '223SLHC', '224HST',
       '225HCT', '226MCV', '227MCH', '228MCHC', '231HCN', '232HCL%',
       '234SLTC', 236, 237, 238, 240, '246SLBC', '256TT', '257A', '258B',
       '259M', '260L', 261, '261%LUC', 262, 263, 264, 265, '750HbA1',
       '751HbA2', '931Tha.CD41/42', '932Tha.CD71/72', '936Tha.CD95',
       '937Tha.CD17', '943HLATuy', '963Tha.Co26', '964Tha. IVSI5',
       '965Tha.SEA', '966Tha.THAI', '967Tha.FIL', '968Tha.Alpha4.2',
       '969Tha.Alpha3.7', '970Tha.IVSI1', '9850HbCs', '9856-28'],
      dtype=object)

In [52]:
feature_df = df[basic_names]
feature_df = feature_df.loc[::-1]
feature_df = feature_df.apply(pd.to_numeric, errors='coerce', downcast='float')
sorted_df = feature_df.loc[feature_df.isnull().sum(axis=1).sort_values().index]
values = sorted_df.iloc[0].to_dict()
obj.update(values)


In [93]:
tgt_feature_names = alpha_names
# tgt_feature_names = beta_names
new_df = df.iloc[::-1]
intersection_columns = set(tgt_feature_names) & set(new_df.columns)
transpose_df = new_df[intersection_columns]
# transpose_df['432THAI'].values[-1] = np.nan
transpose_df.replace(regex='DƯƠNG', value=-2, inplace=True)
transpose_df.replace(regex='ÂM', value=-1, inplace=True)

vl = transpose_df.replace(regex='x', value=np.nan).fillna(method='bfill').iloc[0].to_dict()
obj.update(vl)


/home/tupm/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:4521: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [ ]:
def update_label(df, tgt_feature_names, obj):
    new_df = df.drop(columns=["Tên xét nghiệm"]).set_index("Mã XN").transpose()
    intersection_columns = set(tgt_feature_names) & set(new_df.columns)
    transpose_df = new_df[intersection_columns]
    # mask = transpose_df.replace(regex='DƯƠNG', value=-2).values == -2
    for idx, column in enumerate(transpose_df.columns):
        for i in range(len(transpose_df)-1, -1, -1):
            if "ÂM" in str(transpose_df[column].iloc[i]):
                obj[column] = -1
                break
            
            if "DƯƠNG" in str(transpose_df[column].iloc[i]):
                obj[column] = -2
                break
    
    return len(intersection_columns) != 0